In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-14"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14652,2024-11-14,Brasil Nbb,19:30,Fortaleza B.C.,São Paulo,2.56,1.42,149.5,1.85,1.81,4.5,1.94,1.73,CrbmXAcn,0.390625,0.704225,0.540541,0.552486,0.094850,0.0,0.6,NaN,NaN,212.544,70.183067,0.330205,136.032,53.843671,0.395816,138.192,176.928,154.44,102.12,0.0,0.0,0.0,0.0,0.405076,0.015456,0.080922,-2.66,-0.532,-2.932331,0.627091,0.5,-0.127091,2.38,0.476,0.882353,0.556607,0.6,0.043393
14653,2024-11-14,Brasil Nbb,19:30,União Corinthians,Minas,2.62,1.41,154.5,1.81,1.85,4.5,1.99,1.70,6sidVlSb,0.381679,0.709220,0.552486,0.540541,0.090899,0.4,0.2,NaN,NaN,255.716,136.220686,0.532703,102.204,13.616412,0.133228,173.716,131.976,153.09,126.42,0.0,0.0,0.0,0.0,0.424615,0.015456,0.111144,1.12,0.224,7.232143,0.488116,0.4,-0.088116,0.52,0.104,3.942308,0.694324,0.8,0.105676
14654,2024-11-14,Eua Nba,00:00,Los Angeles Lakers,Memphis Grizzlies,1.35,3.37,231.5,1.93,1.93,-7.5,1.97,1.89,EHXcuxDN,0.740741,0.296736,0.518135,0.518135,0.037477,0.4,0.6,NaN,NaN,168.822,40.689211,0.241018,205.896,41.337512,0.200769,192.294,211.348,145.14,247.90,0.0,0.0,0.0,0.0,0.605235,0.000000,0.029310,2.98,0.596,0.587248,0.575199,0.7,0.124801,0.29,0.058,40.862069,0.527493,0.8,0.272507
14655,2024-11-14,Eua Nba,00:00,Portland Trail Blazers,Minnesota Timberwolves,4.79,1.21,218.5,1.84,2.00,9.5,2.01,1.86,AmJTZFzo,0.208768,0.826446,0.543478,0.500000,0.035215,0.6,0.8,NaN,NaN,367.444,205.877917,0.560297,142.994,19.614476,0.137170,369.504,178.998,494.10,137.16,1.0,0.0,0.0,0.0,0.843814,0.058926,0.054814,2.00,0.400,9.475000,0.377290,0.5,0.122710,-1.76,-0.352,-0.596591,0.565258,0.4,-0.165258
14656,2024-11-14,Eua Nba,00:00,Sacramento Kings,Phoenix Suns,1.32,3.57,224.5,1.81,2.04,-8.5,2.02,1.83,Qe1GcJSi,0.757576,0.280112,0.552486,0.490196,0.037688,0.2,0.8,NaN,NaN,196.296,69.259295,0.352831,192.198,38.758793,0.201661,149.348,215.462,138.18,199.20,0.0,0.0,0.0,0.0,0.650712,0.084485,0.069792,-2.71,-0.542,-0.590406,0.623246,0.4,-0.223246,1.97,0.394,6.522843,0.521433,0.7,0.178567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14743,2024-11-14,Macedônia Do Norte Prva Liga,11:00,Vardar,MZT Skopje,16.00,1.01,156.5,1.83,1.83,22.5,1.83,1.83,KbAMbaA0,0.062500,0.990099,0.546448,0.546448,0.052599,0.6,0.4,NaN,NaN,227.866,153.727670,0.674641,123.900,24.815526,0.200287,382.858,148.330,497.00,148.07,1.0,0.0,0.0,0.0,1.246271,0.000000,0.000000,-3.24,-0.648,-23.148148,0.000000,0.0,0.000000,-1.19,-0.238,-0.042017,0.000000,0.0,0.000000
14744,2024-11-14,Rússia Premier League Feminina,09:00,Enisey Krasnoyarsk F,Dynamo Kursk F,7.39,26.00,136.5,1.86,1.84,17.5,1.97,1.71,rV6bzXc1,0.135318,0.038462,0.537634,0.543478,-0.826220,0.4,0.0,NaN,NaN,151.012,92.212901,0.610633,98.922,32.326537,0.326788,121.282,0.000,117.03,77.77,0.0,0.0,0.0,0.0,0.788215,0.007644,0.099917,1.26,0.252,25.357143,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
14745,2024-11-14,Rússia Premier League Feminina,09:00,Novosibirsk F,Dynamo Moscow F,1.29,3.37,134.5,1.85,1.85,-9.5,2.02,1.61,06wPjhrr,0.775194,0.296736,0.540541,0.540541,0.071930,0.4,0.4,NaN,NaN,402.576,287.626116,0.714464,469.600,475.131327,1.011779,426.598,723.404,187.04,166.10,0.0,0.0,0.0,0.0,0.631237,0.000000,0.159732,-3.46,-0.692,-0.419075,0.000000,0.0,0.000000,-5.00,-1.000,-2.370000,0.000000,0.0,0.000000
14746,2024-11-14,Rússia Premier League

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Los Angeles Lakers,Memphis Grizzlies,231.5,1.93,0.9605,Over
1,00:00,Eua Nba,Portland Trail Blazers,Minnesota Timberwolves,218.5,1.84,0.9738,Over
2,00:00,Eua Nba,Sacramento Kings,Phoenix Suns,224.5,1.81,0.8888,Over
3,11:30,Rússia Liga Vtb United,Parma Perm,Saratov,154.5,1.84,0.9983,Over
4,13:30,Rússia Liga Vtb United,MBA Moscow,Zenit Petersburg,157.5,1.85,0.9974,Over
